# ORA
the general and specfic for KEGG, REACTOME and GO

In [1]:
import sys
import pandas as pd
sys.path.append('/Users/yangxu/PyPathway')

In [2]:
from pypathway import Reactome, GO, KEGG, ORA
from pypathway import ColorectalCancer, IdMapping, GMTUtils
from pypathway import EnrichmentExport

In [3]:
# load a gmt file.
gmt = GMTUtils.parse_gmt_file("../../pypathway/tests/gmt_file/h.all.v6.0.entrez.gmt")

In [4]:
# load the example
c = ColorectalCancer()

In [5]:
# infomation of datasets
len(c.deg_list), len(c.background)

(5320, 17216)

In [6]:
res_h = ORA.run(c.deg_list, c.background, gmt)

In [8]:
res_h.table.head()

,name,mapped,number in study,p-value,fdr
0,HALLMARK_GLYCOLYSIS,184,91,9.956407e-08,3.555860e-07
1,HALLMARK_HEME_METABOLISM,180,86,1.426538e-06,3.962605e-06
2,HALLMARK_HEDGEHOG_SIGNALING,36,16,6.011661e-02,8.588088e-02
3,HALLMARK_XENOBIOTIC_METABOLISM,191,103,2.703391e-11,2.703391e-10
4,HALLMARK_HYPOXIA,180,98,3.951669e-11,2.822620e-10


In [9]:
# inline bar plot.
res_h.plot()

## KEGG

In [11]:
# kegg enrichment for certain organism
r_kg = KEGG.run(c.deg_list, c.background, 'hsa')

In [12]:
r_kg.table.head()

,ID,Name,mapped,deg,p-value,fdr
0,hsa00380,Tryptophan metabolism - Homo sapiens (human),38,19,0.010641,0.043101
1,hsa04911,Insulin secretion - Homo sapiens (human),75,28,0.139888,0.258753
2,hsa01524,Platinum drug resistance - Homo sapiens (human),65,23,0.255051,0.390508
3,hsa00440,Phosphonate and phosphinate metabolism - Homo ...,5,2,0.490263,0.642968
4,hsa00561,Glycerolipid metabolism - Homo sapiens (human),57,33,0.000022,0.000349


In [13]:
r_kg.plot()

## Reactome

In [15]:
# the Example of using the warpper of Reactome gene set enrichment analysis

In [16]:
sybs = [x[1][0] for x in IdMapping.convert(input_id=c.deg_list, organism='hsa', source='ENTREZID', target='SYMBOL') if x[1]]

In [17]:
# the input is a list of symbol
r = Reactome.run(sybs[:10], organism='Homo sapiens')

4


In [19]:
# the result
r.table.head()

,name,dbId,found,p-value,fdr,species
0,Acetylation,156582,2,0.000722,0.031066,Homo sapiens
1,Defective SLC6A2 causes orthostatic intoleranc...,5619109,1,0.007301,0.153318,Homo sapiens
2,Amino acid transport across the plasma membrane,352230,2,0.012057,0.168804,Homo sapiens
3,Neurotransmitter uptake and metabolism In glia...,112313,1,0.024136,0.184060,Homo sapiens
4,Astrocytic Glutamate-Glutamine Uptake And Meta...,210455,1,0.024136,0.184060,Homo sapiens


## Gene ontology

In [21]:
# make the association file using the id_mapping function
# detail using will be shown in the utils section
r = IdMapping.convert_to_dict(input_id=c.background, source='ENTREZID', target="GO", organism='hsa')

In [22]:
# run go enrichment analysis via goatools 
# the inputs of study, pop, and assoc is list, list,  dict.
# the path is the folder of go obo file
path = 'path_to_obo'
rg = GO.run([str(x) for x in c.deg_list], [str(x) for x in c.background], r, obo=path)

load obo file /Users/yangxu/goatools/go-basic.obo
/Users/yangxu/goatools/go-basic.obo: fmt(1.2) rel(2017-04-28) 48,589 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..


15,253 out of 17,216 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
 5,104 out of  5,320 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  15,739 GO terms are associated with 5,104 of 5,320 study items
  20,709 GO terms are associated with 15,253 of 17,216 population items


In [23]:
# the file input of study, pop and assoc
# this function is the warpper of the Goatools
# Github: https://github.com/tanghaibao/goatools
# cite: Haibao Tang et al. (2015). GOATOOLS: Tools for Gene Ontology. Zenodo. 10.5281/zenodo.31628.

path = "path_tO_goatools_dir"
rg = GO.run(path + 'data/study', path + 'data/population', path + 'data/association',
        obo=path + 'go-basic.obo')

load obo file /Users/yangxu/goatools/go-basic.obo
/Users/yangxu/goatools/go-basic.obo: fmt(1.2) rel(2017-04-28) 48,589 GO Terms
fisher module not installed.  Falling back on scipy.stats.fisher_exact


Propagating term counts to parents ..
terms not found: {'GO:0022627', 'GO:0022625'}


31,855 out of 33,239 population items found in association
Calculating uncorrected p-values using fisher_scipy_stats
   269 out of    276 study items found in association
Running multitest correction: local bonferroni
Running multitest correction: local sidak
Running multitest correction: local holm
  797 GO terms are associated with 269 of 276 study items
  6,122 GO terms are associated with 31,855 of 33,239 population items


In [24]:
rg.table.head()

,GO,NS,enrichment,name,ratio_in_study,ratio_in_pop,p_uncorrected,depth,study_count,p_bonferroni,p_sidak,p_holm
0,GO:0036211,BP,e,protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505
1,GO:0006464,BP,e,cellular protein modification process,33/276,1727/33239,0.000008,n.a.,33,0.0505,0.0492,0.0505
2,GO:0006468,BP,e,protein phosphorylation,22/276,922/33239,0.000011,n.a.,22,0.0660,0.0643,0.0660
3,GO:0016310,BP,e,phosphorylation,22/276,996/33239,0.000035,n.a.,22,0.2130,0.2070,0.2120
4,GO:0043412,BP,e,macromolecule modification,33/276,1877/33239,0.000057,n.a.,33,0.3510,0.3420,0.3510


In [25]:
# the interactive graph display the significance of the result by color
rg.graph()

In [26]:
# test export
c = EnrichmentExport.export([rg, rg])